In [1]:
import timeit
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tables as tb

In [2]:
from torch.autograd import Variable
from torch.utils.data import Dataset
from MF_RP_mat_h5support import H5_NODE_NAME
m = nn.Conv2d(16, 33, 3, padding=0)

input = torch.randn(20,16,50,100)
_input = input.numpy()
out = m(input)

In [3]:
pool = nn.MaxPool2d(2, stride=2, return_indices=True)
unpool = nn.MaxUnpool2d(2, stride=2)
input = torch.tensor([[[[ 1.,  2,  3,  4],
                            [ 5,  6,  7,  8],
                            [ 9, 10, 11, 12],
                            [13, 14, 15, 16]]]])
output, indices = pool(input)
a=pool(input)

In [4]:
a = torch.randn((3, 10))
b = torch.randn((3,10))
c = a -b
d= c**2
dT= d.T

In [5]:
#***STUDENT T DISTRIBUTION TEST***
#q_ij = 1/(1+dist(z_i, u_j)^2), then normalize it.
centroids = np.array(
    [[1,1,1,1],
    [3,3,3,3],
    [6,6,6,6]]
) #u, 3 classes, each class centroid is 4D vector
emb1 = np.array([[13,14,15,16],])
emb2 = np.array([[17,18,19,20],])
embs = np.array([emb1, emb2])#z, 2 samples, each sample embedded to 4D vector

z_minus_u = embs - centroids # diff between each z and each centroids
print('centroids(u):')
print(centroids)
print('---')
print('embs(z):')
print(embs)
print('---')
print('z_minus_u:')
print(z_minus_u)
z_minus_u_square = np.square(z_minus_u)  # i.e., distance square between each z and each centroids
dist_square_sum = np.sum(z_minus_u_square, axis=2) # each element: square of euclid distance
print('---')
print('z_minus_u_square:')
print(z_minus_u_square)
print('---')
print('dist_square_sum:')
print(dist_square_sum)
alpha = 1.
qij = 1. / (1. + dist_square_sum / 1.) # each element: the probability of sample i belong to j
print('---')
print('qij:')
print(qij)
qij_sum = np.sum(qij, axis=1) # sum probability to all 3 centroid
print('---')
print('qij_sum:')
print(qij_sum)
qij_t = np.transpose(qij)
print('---')
print('qij_t:')
print(qij_t)
qij_normalize = qij_t / qij_sum # each probability/sum probability
print('---')
print('qij_normalize:')
print(qij_normalize)
print('---')
print('qij_normalize_t:') # note !!qij_normalize_t each row sum to 1, which is suitable for kl_div loss
print(qij_normalize.T) # transpose back to denote the probability of i belong to j

centroids(u):
[[1 1 1 1]
 [3 3 3 3]
 [6 6 6 6]]
---
embs(z):
[[[13 14 15 16]]

 [[17 18 19 20]]]
---
z_minus_u:
[[[12 13 14 15]
  [10 11 12 13]
  [ 7  8  9 10]]

 [[16 17 18 19]
  [14 15 16 17]
  [11 12 13 14]]]
---
z_minus_u_square:
[[[144 169 196 225]
  [100 121 144 169]
  [ 49  64  81 100]]

 [[256 289 324 361]
  [196 225 256 289]
  [121 144 169 196]]]
---
dist_square_sum:
[[ 734  534  294]
 [1230  966  630]]
---
qij:
[[0.00136054 0.00186916 0.00338983]
 [0.00081235 0.00103413 0.00158479]]
---
qij_sum:
[0.00661953 0.00343126]
---
qij_t:
[[0.00136054 0.00081235]
 [0.00186916 0.00103413]
 [0.00338983 0.00158479]]
---
qij_normalize:
[[0.20553476 0.2367491 ]
 [0.28237018 0.3013838 ]
 [0.51209507 0.4618671 ]]
---
qij_normalize_t:
[[0.20553476 0.28237018 0.51209507]
 [0.2367491  0.3013838  0.4618671 ]]


In [6]:
#How to calculate the mean and std of my own dataset?
#https://discuss.pytorch.org/t/about-normalization-using-pre-trained-vgg16-networks/23560/23
class MyDataset(Dataset):
    def __init__(self):
        self.data = torch.randn(1000, 3, 224, 224)

    def __getitem__(self, index):
        x = self.data[index]
        return x

    def __len__(self):
        return len(self.data)

def main():
    device = torch.device("cpu")
    dataset = MyDataset()

    start = timeit.time.perf_counter()
    data = dataset.data.to(device)
    print("Mean:", torch.mean(data, dim=(0, 2, 3))) # get mean on each channel for all samples
    print("Std:", torch.std(data, dim=(0, 2, 3)))
    print("Elapsed time: %.3f seconds" % (timeit.time.perf_counter() - start))
    print()

    start = timeit.time.perf_counter()
    mean = 0.
    for data in dataset:
        data = data.to(device)
        mean += torch.mean(data, dim=(1, 2))
    mean /= len(dataset)
    print("Mean:", mean)

    temp = 0.
    nb_samples = 0.
    for data in dataset:
        data = data.to(device)
        temp += ((data.view(3, -1) - mean.unsqueeze(1)) ** 2).sum(dim=1)
        nb_samples += np.prod(data.size()[1:])
    std = torch.sqrt(temp/nb_samples)
    print("Std:", std)
    print("Elapsed time: %.3f seconds" % (timeit.time.perf_counter() - start))
# main()

In [7]:
# data_type = 'train'
# RP_mats_h5array = tb.open_file(f'./data/SHL_features/RP_mats_{data_type}.h5', mode='r').get_node('/' + H5_NODE_NAME)
# RP_mats_h5array.shape
# e=RP_mats_h5array[:]

In [8]:
x=torch.arange(24, dtype=torch.float).view(2,3,4)
print(x)
x_mean0=torch.mean(x,dim=(0,1),keepdim=False)
x_mean1=torch.mean(x,dim=0,keepdim=False)
x_mean2=torch.mean(x,dim=1,keepdim=False)
print(x_mean0)
print(x_mean1)
print(x_mean2)

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])
tensor([10., 11., 12., 13.])
tensor([[ 6.,  7.,  8.,  9.],
        [10., 11., 12., 13.],
        [14., 15., 16., 17.]])
tensor([[ 4.,  5.,  6.,  7.],
        [16., 17., 18., 19.]])


In [9]:
loss = torch.nn.MSELoss(size_average=True)
input = Variable(torch.ones(2,2), requires_grad=True)
target = Variable(torch.Tensor([[3,3],[3,3]]))
output = loss(input, target)
output.backward()
gra=input.grad

c:\users\lsc\anaconda3\envs\p37\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [10]:
a = torch.randn((3, 10))
b = torch.randn((3,10))
c = a -b
d= c**2
dT= d.T

In [11]:
#***STUDENT T DISTRIBUTION TEST 2***
#q_ij = 1/(1+dist(z_i, u_j)^2), then normalize it.
centroids = np.array(
    [[1,1,1,1],
    [3,3,3,3],
    [6,6,6,6]]
) #u, 3 classes, each class centroid is 4D vector
centroids = np.array([centroids, centroids]) # !note: repeated 2 times
emb1 = np.array([[13,14,15,16],])
emb2 = np.array([[17,18,19,20],])
embs = np.array([emb1, emb2])#z, 2 samples, each sample embedded to 4D vector

z_minus_u = embs - centroids # diff between each z and each centroids
print('centroids(u):')
print(centroids)
print('---')
print('embs(z):')
print(embs)
print('---')
print('z_minus_u:')
print(z_minus_u)
z_minus_u_square = np.square(z_minus_u)  # i.e., distance square between each z and each centroids
dist_square_sum = np.sum(z_minus_u_square, axis=2) # each element: square of euclid distance
print('---')
print('z_minus_u_square:')
print(z_minus_u_square)
print('---')
print('dist_square_sum:')
print(dist_square_sum)
alpha = 1.
qij = 1. / (1. + dist_square_sum / 1.) # each element: the probability of sample i belong to j
print('---')
print('qij:')
print(qij)
qij_sum = np.sum(qij, axis=1) # sum probability to all 3 centroid
print('---')
print('qij_sum:')
print(qij_sum)
qij_t = np.transpose(qij)
print('---')
print('qij_t:')
print(qij_t)
qij_normalize = qij_t / qij_sum # each probability/sum probability
print('---')
print('qij_normalize:')
print(qij_normalize)
print('---')
print('qij_normalize_t:')
print(qij_normalize.T) # transpose back to denote the probability of i belong to j

centroids(u):
[[[1 1 1 1]
  [3 3 3 3]
  [6 6 6 6]]

 [[1 1 1 1]
  [3 3 3 3]
  [6 6 6 6]]]
---
embs(z):
[[[13 14 15 16]]

 [[17 18 19 20]]]
---
z_minus_u:
[[[12 13 14 15]
  [10 11 12 13]
  [ 7  8  9 10]]

 [[16 17 18 19]
  [14 15 16 17]
  [11 12 13 14]]]
---
z_minus_u_square:
[[[144 169 196 225]
  [100 121 144 169]
  [ 49  64  81 100]]

 [[256 289 324 361]
  [196 225 256 289]
  [121 144 169 196]]]
---
dist_square_sum:
[[ 734  534  294]
 [1230  966  630]]
---
qij:
[[0.00136054 0.00186916 0.00338983]
 [0.00081235 0.00103413 0.00158479]]
---
qij_sum:
[0.00661953 0.00343126]
---
qij_t:
[[0.00136054 0.00081235]
 [0.00186916 0.00103413]
 [0.00338983 0.00158479]]
---
qij_normalize:
[[0.20553476 0.2367491 ]
 [0.28237018 0.3013838 ]
 [0.51209507 0.4618671 ]]
---
qij_normalize_t:
[[0.20553476 0.28237018 0.51209507]
 [0.2367491  0.3013838  0.4618671 ]]


In [12]:
# kld test, this is the same example in wiki
P = torch.Tensor([0.36, 0.48, 0.16]) #traget
Q = torch.Tensor([0.333, 0.333, 0.333]) #pred
print(
(P * (P / Q).log()).sum())
# tensor(0.0863), 10.2 µs ± 508
a = (0.36 * np.log(.36/.333) + 0.48 * np.log(.48/.333) + 0.16 * np.log(.16/.333))
print(a)
print(F.kl_div(Q.log(), P, None, None, 'sum'))
# tensor(0.0863), 14.1 µs ± 408 ns



tensor(0.0863)
0.08630010165195404
tensor(0.0863)


In [13]:
# kld test
# https://github.com/pytorch/pytorch/pull/14457/files
# https://github.com/pytorch/pytorch/issues/6622
input_shape = (2, 5)
log_prob1 = F.log_softmax(torch.randn(input_shape), 1)
print(log_prob1)
prob2 = F.softmax(torch.randn(input_shape), 1)
loss = nn.KLDivLoss(reduction='mean') # wrong
# loss = nn.KLDivLoss(reduction='batchmean') # right
l = loss(log_prob1, prob2)
loss_none_reduce = nn.KLDivLoss(reduction='sum')(log_prob1, prob2)
expected = loss_none_reduce / input_shape[0]
wrong_l = loss_none_reduce / (2*5) # element-wise indeed, which is wrong for kld loss
l, expected, wrong_l


tensor([[-3.1285, -1.7640, -1.6139, -4.0444, -0.5653],
        [-2.5421, -2.2013, -1.4105, -0.7695, -2.2693]])


c:\users\lsc\anaconda3\envs\p37\lib\site-packages\torch\nn\functional.py:2352: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


(tensor(0.1329), tensor(0.6643), tensor(0.1329))

In [15]:
# loss function weight test
x1 = torch.tensor(1.0, requires_grad=True)
y1 = torch.tensor(2.0, requires_grad=True)
z1 = x1**2+y1

x2 = torch.tensor(3.0, requires_grad=True)
y2 = torch.tensor(4.0, requires_grad=True)
z2 = x2**3+2*y2

x3 = torch.tensor(5.0, requires_grad=True)
y3 = torch.tensor(6.0, requires_grad=True)
z3 = x3**4+3*y3

alpha = 1
beta = 1
gamma = 1

Z = alpha*z1 + beta*z2 + gamma*z3
Z.backward()
print(Z, x1.grad, x2.grad, x3.grad)

tensor(73., grad_fn=<AddBackward0>) tensor(2.) tensor(54.) None
